In [1]:
!pip install tensorflow
!pip install nltk

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle (2).json to kaggle (2) (1).json


In [11]:
import os
# Make a directory for kaggle.json
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json to the correct location
!mv kaggle.json /root/.kaggle/

# Change the permissions of the file to make it readable
!chmod 600 /root/.kaggle/kaggle.json




In [12]:
!kaggle datasets list



ref                                                         title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
bhadramohit/customer-shopping-latest-trends-dataset         Customer Shopping (Latest Trends) Dataset      76KB  2024-11-23 15:26:12           8054        148  1.0              
malaiarasugraj/global-health-statistics                     Global Health Statistics                       44MB  2024-11-27 10:52:27           2136         29  1.0              
mujtabamatin/air-quality-and-pollution-assessment           Air Quality and Pollution Assessment           84KB  2024-12-04 15:29:51           2461         45  1.0              
hopesb/student-depression-dataset                           Student Depression Dataset.                   454K

In [13]:
!kaggle datasets download -d nikhiljohnk/tweets-with-sarcasm-and-irony


Dataset URL: https://www.kaggle.com/datasets/nikhiljohnk/tweets-with-sarcasm-and-irony
License(s): CC0-1.0
  0% 0.00/4.27M [00:00<?, ?B/s]
100% 4.27M/4.27M [00:00<00:00, 59.8MB/s]


In [14]:
!unzip tweets-with-sarcasm-and-irony.zip -d /content/dataset/


Archive:  tweets-with-sarcasm-and-irony.zip
  inflating: /content/dataset/test.csv  
  inflating: /content/dataset/train.csv  


In [16]:
!ls /content/dataset/



test.csv  train.csv


In [17]:
import pandas as pd

# Load the training and testing datasets
train_df = pd.read_csv('/content/dataset/train.csv')
test_df = pd.read_csv('/content/dataset/test.csv')

# Preview the first few rows of the training and test datasets
print("Training Data:")
print(train_df.head())

print("\nTesting Data:")
print(test_df.head())


Training Data:
                                              tweets       class
0  Be aware  dirty step to get money  #staylight ...  figurative
1  #sarcasm for #people who don't understand #diy...  figurative
2  @IminworkJeremy @medsingle #DailyMail readers ...  figurative
3  @wilw Why do I get the feeling you like games?...  figurative
4  -@TeacherArthurG @rweingarten You probably jus...  figurative

Testing Data:
                                              tweets       class
0  no one ever predicted this was going to happen...  figurative
1  @Stooshie its as closely related as Andrews or...  figurative
2  I find it ironic when Vegans say they love foo...  figurative
3  Quick rt that throwing money vine I've not see...  figurative
4  yep, keep adding me to your #devops lists.... ...  figurative


In [18]:
print("Train Data Columns:", train_df.columns)
print("Test Data Columns:", test_df.columns)

# Check for missing values
print("\nMissing Values in Train Data:")
print(train_df.isnull().sum())

print("\nMissing Values in Test Data:")
print(test_df.isnull().sum())


Train Data Columns: Index(['tweets', 'class'], dtype='object')
Test Data Columns: Index(['tweets', 'class'], dtype='object')

Missing Values in Train Data:
tweets    0
class     0
dtype: int64

Missing Values in Test Data:
tweets    2
class     9
dtype: int64


In [19]:
# Fill missing values in 'tweets' with an empty string
test_df['tweets'].fillna('', inplace=True)

# Drop rows with missing 'class' labels
test_df.dropna(subset=['class'], inplace=True)

# Verify that missing values are handled
print("\nMissing Values in Test Data after handling:")
print(test_df.isnull().sum())



Missing Values in Test Data after handling:
tweets    0
class     0
dtype: int64


<ipython-input-19-ff50232e306d>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_df['tweets'].fillna('', inplace=True)


In [20]:
import re
import nltk
from nltk.tokenize import word_tokenize

# Ensure you have the NLTK data files
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    return ' '.join(tokens)

In [31]:
import nltk
nltk.data.path.append('/content/nltk_data')  # Optional: Set custom path
nltk.download('punkt', force=True)  # Force re-download



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [30]:
import nltk
print(nltk.data.find('tokenizers/punkt'))


/root/nltk_data/tokenizers/punkt


In [34]:
import nltk
import shutil

# Delete existing punkt data
punkt_path = '/root/nltk_data/tokenizers/punkt'
shutil.rmtree(punkt_path, ignore_errors=True)

# Redownload punkt
nltk.download('punkt', download_dir='/root/nltk_data')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [35]:
import nltk

nltk.data.path.append('/root/nltk_data')
nltk.download('punkt', download_dir='/root/nltk_data')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
import re

def preprocess_text(text):
    # Simple regex-based tokenization
    tokens = re.findall(r'\b\w+\b', text.lower())
    return " ".join(tokens)


In [37]:
train_df['cleaned_text'] = train_df['tweets'].apply(preprocess_text)
test_df['cleaned_text'] = test_df['tweets'].apply(preprocess_text)

In [39]:
print("Processed Training Data:")
print(train_df[['tweets', 'cleaned_text']].head())

print("Processed Testing Data:")
print(test_df[['tweets', 'cleaned_text']].head())

Processed Training Data:
                                              tweets  \
0  Be aware  dirty step to get money  #staylight ...   
1  #sarcasm for #people who don't understand #diy...   
2  @IminworkJeremy @medsingle #DailyMail readers ...   
3  @wilw Why do I get the feeling you like games?...   
4  -@TeacherArthurG @rweingarten You probably jus...   

                                        cleaned_text  
0  be aware dirty step to get money staylight sta...  
1  sarcasm for people who don t understand diy ar...  
2  iminworkjeremy medsingle dailymail readers bei...  
3  wilw why do i get the feeling you like games s...  
4  teacherarthurg rweingarten you probably just m...  
Processed Testing Data:
                                              tweets  \
0  no one ever predicted this was going to happen...   
1  @Stooshie its as closely related as Andrews or...   
2  I find it ironic when Vegans say they love foo...   
3  Quick rt that throwing money vine I've not see...   
4  y

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 5000
max_sequence_length = 100

In [41]:
tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
tokenizer.fit_on_texts(train_df['cleaned_text'])

In [42]:
X_train = tokenizer.texts_to_sequences(train_df['cleaned_text'])
X_test = tokenizer.texts_to_sequences(test_df['cleaned_text'])

In [43]:
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

In [44]:
y_train = train_df['class']
y_test = test_df['class']

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D, Bidirectional

In [78]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100))
model.add(SpatialDropout1D(0.2))
model.add((LSTM(64, dropout=0.3, recurrent_dropout=0.2)))
model.add(Dense(4, activation='softmax'))

In [79]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Split your data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(train_df['tweets'], train_df['class'], test_size=0.2, random_state=42)

# Convert string labels to integers using LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the labels for training set
y_train = label_encoder.fit_transform(y_train)

# Transform the validation labels using the fitted encoder
y_val = label_encoder.transform(y_val)

# One-hot encode the labels for multi-class classification
y_train = to_categorical(y_train, num_classes=4)  # Adjust num_classes as per your dataset
y_val = to_categorical(y_val, num_classes=4)

# Tokenize the text data for input to the LSTM model (using the same tokenizer)
tokenizer = Tokenizer(num_words=max_words, lower=True, split=' ')
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_val = pad_sequences(X_val, maxlen=max_sequence_length)

Exception ignored in: <function _xla_gc_callback at 0x79114be33d90>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [81]:

history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_val, y_val), verbose=1)

Epoch 1/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 172s 165ms/step - accuracy: 0.6774 - loss: 0.6892 - val_accuracy: 0.7428 - val_loss: 0.4982
Epoch 2/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 199s 163ms/step - accuracy: 0.7387 - loss: 0.4989 - val_accuracy: 0.7428 - val_loss: 0.4997
Epoch 3/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 160s 158ms/step - accuracy: 0.7382 - loss: 0.4955 - val_accuracy: 0.7423 - val_loss: 0.5023
Epoch 4/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 177s 173ms/step - accuracy: 0.7421 - loss: 0.4876 - val_accuracy: 0.7347 - val_loss: 0.5077
Epoch 5/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 172s 169ms/step - accuracy: 0.7428 - loss: 0.4827 - val_accuracy: 0.7265 - val_loss: 0.5203


Epoch 1/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 398s 384ms/step - accuracy: 0.6817 - loss: 0.6718 - val_accuracy: 0.7426 - val_loss: 0.5012
Epoch 2/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 446s 388ms/step - accuracy: 0.7355 - loss: 0.5019 - val_accuracy: 0.7424 - val_loss: 0.5013
Epoch 3/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 419s 411ms/step - accuracy: 0.7360 - loss: 0.4969 - val_accuracy: 0.7411 - val_loss: 0.5041
Epoch 4/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 457s 426ms/step - accuracy: 0.7408 - loss: 0.4906 - val_accuracy: 0.7344 - val_loss: 0.5121
Epoch 5/5
1018/1018 ━━━━━━━━━━━━━━━━━━━━ 435s 420ms/step - accuracy: 0.7417 - loss: 0.4852 - val_accuracy: 0.7346 - val_loss: 0.5196
